In [1]:
!pip install pyDOE
!pip install cma


  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=3b11b895e59220861f4c5c9ea142a4199a09bdae606ffdcb1862e1d5b788dd5c
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 2.8MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import PolynomialFeatures
import bbobbenchmarks as bn
import sklearn

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F15 '''
def F15(X):
    f = bn.F15()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data

''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))
      

''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 0.1404 ,random_state=0 , l1_ratio=0.8282, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 63.9 , C = 850.81 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_15_4000Samples.csv"
train_y = pd.read_csv(path).iloc[:,-1]
test_y = pd.read_csv('test_15_800Samples.csv').iloc[:,-1]
path = "train_latent_200D.csv"
train = pd.read_csv(path, index_col = 0)
test = pd.read_csv('test_latent_200D.csv', index_col = 0)
train ['Y'] = train_y
test ['Y'] = test_y
del train_y
del test_y
true = np.array(test['Y'])

## Surrogate Models

In [5]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,101):
        Cols.append(Columns[j]+'_Z'+str(i))

opt = cma.CMAOptions()
opt.set ("seed" , 0)
opt.set ("maxfevals" , 100000)

n_obs , dim =  30, 100
G = DOE(n_obs, dim)
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([30,300])
for i in range(G[1].shape[0]):
    print ('Run : '+ str(i))
    min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 0.1566 , options=opt) [0]
    min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 0.1566 , options=opt) [0]
    min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 0.1566 , options=opt) [0]
    X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 0
(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=864289, Mon Apr 13 17:59:02 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 2.064648467477377e+04 1.0e+00 1.51e-01  2e-01  2e-01 0:00.0
termination on tolfun=1e-11 (Mon Apr 13 17:59:02 2020)
final/bestever f-value = 2.064648e+04 2.064648e+04
incumbent solution: [0.79928062 0.96511706 0.27511535 0.55431261 0.38598683 0.54448499
 0.04475447 0.38653623 ...]
std deviations: [0.15071372 0.15084505 0.15070839 0.15081632 0.15073189 0.15077409
 0.15079579 0.15074415 ...]
(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=938845, Mon Apr 13 17:59:02 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 2.008457006765140e+04 1.0e+00 1.50e-01  2e-01  2e-01 0:00.0


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [20646.48467477] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=20646.484675, sigma=1.51e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=20646.484675, sigma=1.51e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  '

termination on tolfun=1e-11 (Mon Apr 13 17:59:02 2020)
final/bestever f-value = 2.008457e+04 2.008457e+04
incumbent solution: [0.80325542 1.02547883 0.23287499 0.73070729 0.46376202 0.53627661
 0.0339737  0.50380937 ...]
std deviations: [0.15040883 0.1505204  0.15040222 0.15044128 0.15045459 0.15032538
 0.15037373 0.15037197 ...]
(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=893010, Mon Apr 13 17:59:02 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 2.428623319829688e+04 1.0e+00 1.51e-01  2e-01  2e-01 0:00.0
    2     34 2.376080331951477e+04 1.0e+00 1.46e-01  1e-01  1e-01 0:00.1
    3     51 2.220664919949958e+04 1.0e+00 1.42e-01  1e-01  1e-01 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [28170.84492357] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [24145.3516071] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


   86   1462 -3.689935439485648e+06 1.3e+00 2.07e+00  2e+00  2e+00 0:03.1
  100   1700 -1.330106039325915e+07 1.3e+00 3.98e+00  4e+00  4e+00 0:03.6
  179   3043 -2.563318986554696e+10 1.3e+00 1.61e+02  2e+02  2e+02 0:06.5
termination on tolfacupx=1000.0 (Mon Apr 13 17:59:10 2020)
final/bestever f-value = -2.530787e+10 -2.563319e+10
incumbent solution: [-568.66081982  164.0109549   128.8091055   140.03353701 -378.22001912
 -621.43500063 -361.23289998  -61.95173304 ...]
std deviations: [158.5709495  158.50010792 156.82072    156.81388567 160.2424159
 159.9223549  158.17128753 157.88083266 ...]
Run : 1
(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=847493, Mon Apr 13 17:59:10 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 2.064648467477377e+04 1.0e+00 1.51e-01  2e-01  2e-01 0:00.0
termination on tolfun=1e-11 (Mon Apr 13 17:59:10 2020)
final/bestever f-value = 2.064648e+04 2.064648e+04
incumbent solution: [ 0.07603347  0.24888359  0.7

...

   86   1462 -4.350708981583366e+06 1.3e+00 1.95e+00  2e+00  2e+00 0:03.1
  100   1700 -1.476971342165431e+07 1.3e+00 3.85e+00  4e+00  4e+00 0:03.6
  182   3094 -3.112086476752665e+10 1.3e+00 1.62e+02  2e+02  2e+02 0:06.6
termination on tolfacupx=1000.0 (Mon Apr 13 18:02:40 2020)
final/bestever f-value = -3.020513e+10 -3.112086e+10
incumbent solution: [-1056.65246093   815.90093368   394.14768405   516.52661934
   -16.27052986  -369.56801338  -373.82727495  -132.51593917 ...]
std deviations: [159.75426549 160.95671605 159.81825096 159.47980167 160.55896511
 159.74259337 159.74956935 160.38671984 ...]
Run : 29
(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=891254, Mon Apr 13 18:02:40 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 2.064648467477377e+04 1.0e+00 1.50e-01  2e-01  2e-01 0:00.0
termination on tolfun=1e-11 (Mon Apr 13 18:02:40 2020)
final/bestever f-value = 2.064648e+04 2.064648e+04
incumbent solution: [0.04516899 0.86413

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [20646.48467477] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=20646.484675, sigma=1.50e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=20646.484675, sigma=1.50e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  '

termination on tolfun=1e-11 (Mon Apr 13 18:02:40 2020)
final/bestever f-value = 2.008457e+04 2.008457e+04
incumbent solution: [0.13445266 0.87651992 0.80174281 0.85577679 0.26507327 0.06061974
 0.36294567 0.475597   ...]
std deviations: [0.15058956 0.15049024 0.15055012 0.15056666 0.15053778 0.15061658
 0.15051401 0.15049491 ...]
(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=866468, Mon Apr 13 18:02:41 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 2.799254672794591e+04 1.0e+00 1.51e-01  2e-01  2e-01 0:00.0
    2     34 2.566425074883230e+04 1.0e+00 1.46e-01  1e-01  1e-01 0:00.1
    3     51 2.290175054008000e+04 1.0e+00 1.43e-01  1e-01  1e-01 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [29657.21603958] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [28810.28589417] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


   86   1462 -6.632282008457330e+06 1.3e+00 2.47e+00  2e+00  2e+00 0:03.1
  100   1700 -2.366882676626629e+07 1.3e+00 4.71e+00  5e+00  5e+00 0:03.6
  180   3060 -2.764845904122013e+10 1.3e+00 1.62e+02  2e+02  2e+02 0:06.6
termination on tolfacupx=1000.0 (Mon Apr 13 18:02:48 2020)
final/bestever f-value = -2.680376e+10 -2.764846e+10
incumbent solution: [-893.43168893  849.11384048  -92.29379031  232.971882   -559.06706739
 -145.80439623 -358.58898877 -102.41479984 ...]
std deviations: [161.75353729 160.63175613 160.44176648 160.67130332 160.45759184
 159.93505383 158.7774392  159.92824424 ...]


In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('F15_200_X_Values.csv')
Krig_Fun = np.zeros(30)
SVM_Fun = np.zeros(30)
ELN_Fun = np.zeros(30)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F15(X_Values.iloc[i,:100])
    SVM_Fun [i] = F15(X_Values.iloc[i,100:200])
    ELN_Fun [i] = F15(X_Values.iloc[i,200:300])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([4030.65546533]), count=array([1])) 133.22414036052368
SVM
ModeResult(mode=array([3950.65149958]), count=array([1])) 145.570600564213
ELN
ModeResult(mode=array([7.76343425e+35]), count=array([1])) 1.1391423415646396e+53
